In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.svm import SVC
from sklearn import grid_search

In [2]:
df = pd.read_csv("titanic_train.csv")
test_df = pd.read_csv("titanic_test.csv")

In [3]:
df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df = df.drop(["Name", "Ticket"], axis=1)
df["Age"] = df["Age"].fillna(df["Age"].median())
df["Embarked"] = df["Embarked"].fillna("S")
test_df = test_df.drop(["Name", "Ticket"], axis=1)
test_df["Age"] = test_df["Age"].fillna(test_df["Age"].median())
test_df["Embarked"] = test_df["Embarked"].fillna("S")
test_df["Fare"] = test_df["Fare"].fillna(test_df["Fare"].median())

In [6]:
df["Cabin"].loc[pd.isnull(df["Cabin"])] = 0
df["Cabin"].loc[df["Cabin"] != 0] = 1
test_df["Cabin"].loc[pd.isnull(test_df["Cabin"])] = 0
test_df["Cabin"].loc[test_df["Cabin"] != 0] = 1

C:\Users\arpit\Anaconda3\lib\site-packages\pandas\core\indexing.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [7]:
df["Sex"].loc[df["Sex"] == "male"] = 1
df["Sex"].loc[df["Sex"] == "female"] = 0
test_df["Sex"].loc[test_df["Sex"] == "male"] = 1
test_df["Sex"].loc[test_df["Sex"] == "female"] = 0

C:\Users\arpit\Anaconda3\lib\site-packages\pandas\core\indexing.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [8]:
df["isChild"] = 0
df["isChild"].loc[df["Age"] <= 18] = 1
test_df["isChild"] = 0
test_df["isChild"].loc[test_df["Age"] <= 18] = 1

C:\Users\arpit\Anaconda3\lib\site-packages\pandas\core\indexing.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [9]:
df["Age<=20"] = 0
df["Age<=20"].loc[df["Age"] <= 20] = 1
df["20<Age<=40"] = 0
df["20<Age<=40"].loc[(df["Age"] > 20) & (df["Age"] <= 40)] = 1
df["Age>40"] = 0
df["Age>40"].loc[df["Age"] > 40] = 1
test_df["Age<=20"] = 0
test_df["Age<=20"].loc[test_df["Age"] <= 20] = 1
test_df["20<Age<=40"] = 0
test_df["20<Age<=40"].loc[(test_df["Age"] > 20) & (test_df["Age"] <= 40)] = 1
test_df["Age>40"] = 0
test_df["Age>40"].loc[test_df["Age"] > 40] = 1

C:\Users\arpit\Anaconda3\lib\site-packages\pandas\core\indexing.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [10]:
df["FamilyMem"] = df["SibSp"] + df["Parch"] + 1
test_df["FamilyMem"] = test_df["SibSp"] + test_df["Parch"] + 1
family_dummy = pd.get_dummies(df["FamilyMem"])
family_dummy.drop(list(family_dummy.keys())[4:], inplace=True, axis=1)
df = df.join(family_dummy)
family_dummy = pd.get_dummies(test_df["FamilyMem"])
family_dummy.drop(list(family_dummy.keys())[4:], inplace=True, axis=1)
test_df = test_df.join(family_dummy)

In [11]:
embarked = pd.get_dummies(df["Embarked"])
df.drop("Embarked", axis=1, inplace=True)
df = df.join(embarked)

embarked = pd.get_dummies(test_df["Embarked"])
test_df.drop("Embarked", axis=1, inplace=True)
test_df = test_df.join(embarked)

In [12]:
df.drop("PassengerId", axis=1, inplace=True)
pass_id = test_df["PassengerId"]
test_df.drop("PassengerId", axis=1, inplace=True)
target = df["Survived"]
data = df.drop("Survived", axis=1)

In [13]:
df.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,isChild,Age<=20,20<Age<=40,Age>40,FamilyMem,1,2,3,4,C,Q,S
0,0,3,1,22.0,1,0,7.2500,0,0,0,1,0,2,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,1,1,0,38.0,1,0,71.2833,1,0,0,1,0,2,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,1,3,0,26.0,0,0,7.9250,0,0,0,1,0,1,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1,1,0,35.0,1,0,53.1000,1,0,0,1,0,2,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,0,3,1,35.0,0,0,8.0500,0,0,0,1,0,1,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [16]:
x = df["Sex"][df["Survived"] == 1]
y = df["Age"][df["Survived"] == 1]
plt.scatter(x, y)
plt.show()

In [20]:
'''
test_df.to_csv("test_csv.csv")
svm = SVC()
param = {"kernel":['linear', 'poly', 'rbf', 'sigmoid'], 'degree':[2, 3]}
svm = grid_search.GridSearchCV(svm, param, cv=5)
svm.fit(data, target)

Y = svm.predict(test_df)
solution = pd.DataFrame({"PassengerID": pass_id, "Survived":Y})
solution.to_csv("solution.csv", index=False)
'''

'\ntest_df.to_csv("test_csv.csv")\nsvm = SVC()\nparam = {"kernel":[\'linear\', \'poly\', \'rbf\', \'sigmoid\'], \'degree\':[2, 3]}\nsvm = grid_search.GridSearchCV(svm, param, cv=5)\nsvm.fit(data, target)\n\nY = svm.predict(test_df)\nsolution = pd.DataFrame({"PassengerID": pass_id, "Survived":Y})\nsolution.to_csv("solution.csv", index=False)\n'

In [19]:
"This solution gives accuracy of 0.77990 on Kaggle."

'This solution gives accuracy of 0.77990 on Kaggle.'